# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Use Pydantic to create a JSON for 

Pydantic is a 'data validation library' for python.
- Works with python type annotations. But rather than static type checking, they are actively used at runtime for data validation and conversion.
- Provides built-in methods to
serialize/deserialize models to/from JSON, dictionaries, etc.
- LangChain leverages Pydantic to create JSON
Scheme describing function.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
foo = User(name="Joe",age=32, email="joe@gmail.com")
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")
foo.age

'bar'

Do it in Pydantic's way

In [10]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [11]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [12]:
foo_p.name

'Jane'

**Note**: The next cell is expected to fail due to its type 

In [13]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [15]:
class Class(BaseModel):
    students: List[pUser]
        
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Use Pydantic to OpenAI function definition


In [38]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

# Field is frrm Pydantic
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    
    # Sai: You don't have to use the Field() to make it mandatory 
    airport_code: str = Field(description="airport code to get weather for")

In [29]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

**Note**: The next cell is expected to generate an error because the docstring of the function is mandatory.

In [34]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

## OpenAI with LCEL

### Pass function as parameter

In [39]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()

model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

### Pass function as bind

In [44]:
# If you bind the model with function, you don't need to invoke(functions=[weather_function]) every time
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Forcing it to use a function

We can force the model to use a function

In [45]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [46]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Using in a chain with prompt and model_with_function

We can use this model bound to function in a chain as we normally would

In [47]:
from langchain.prompts import ChatPromptTemplate

In [51]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])
chain = prompt | model_with_function
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [54]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")
        
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

model_with_functions = model.bind(functions=functions)

In [55]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [56]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"artist_name":"Taylor Swift","n":3}'}})

In [57]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')